In [1]:
import os
import numpy as np

In [2]:
# classes_name =  ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']
classes_name =  ['Car', 'Pedestrian']

classes_num = {}
for idx, class_name in enumerate(classes_name):
    classes_num[class_name] = idx

# classes_num = {'Car': 0, 'Van': 1, 'Truck': 2, 'Pedestrian': 3,
#                'Person_sitting': 4, 'Cyclist': 5, 'Tram': 6, 'Misc': 7}

truncate_th = 0.5
occlude_th = 2

In [3]:
YOLO_ROOT = os.path.abspath('./')
IMAGE_PATH = os.path.join(YOLO_ROOT, 'kitti/data_object_image_2/training/image_2')
DATA_PATH = os.path.join(YOLO_ROOT, 'kitti/training/label_2')
OUTPUT_TRAIN_PATH = os.path.join(YOLO_ROOT, 'kitti_train.txt')

In [4]:
def convert_to_string(image_path, labels):
    """convert image_path, lables to string 
    Returns:
    string 
    """
    out_string = ''
    out_string += image_path
    for label in labels:
        for i in label:
            out_string += ' ' + str(i)
    out_string += '\n'
    return out_string

In [5]:
train_out_file = open(OUTPUT_TRAIN_PATH, 'w')

In [6]:
effect_img = 0
effect_obj = 0
max_obj = 0
for (dirpath, dirnames, filenames) in os.walk(DATA_PATH):
    for file_idx, filename in enumerate(filenames):
        image_filename = filename.replace('txt', 'png')
        image_path = os.path.join(IMAGE_PATH, image_filename)
        
        filepath= os.path.join(dirpath, filename)
        
        with open(filepath) as label_file:
            objs = label_file.readlines()
        
        formatted_objs = []
        for obj in objs:
            [klass, truncated, occluded, _, xmin, ymin, xmax, ymax, _, _, _, _, _, _, _] = obj.split(" ")
            if klass not in classes_name:
                continue
            if float(truncated) > truncate_th:
                continue
            if int(occluded) >= occlude_th:
                continue
            klass_idx = classes_num[klass]
            formatted_objs.append([xmin, ymin, xmax, ymax, str(klass_idx)])
            effect_obj += 1
        
        if len(formatted_objs) == 0:
            continue
        
        if len(formatted_objs) > max_obj:
            max_obj = len(formatted_objs)
        
        effect_img += 1
        
        record = convert_to_string(image_path, formatted_objs)
        train_out_file.write(record)

print("Effective images: " + str(effect_img))
print("Effective objects: " + str(effect_obj))
print("Max objects in : " + str(max_obj))


Effective images: 7471
Effective objects: 23334


In [7]:
train_out_file.close()